In [2]:
import miditok
import pretty_midi
import pandas as pd
import json
import glob
import os

In [3]:
def extract_guitar_and_drums(midi_file):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""    
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))
        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))
    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]
    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    
    return song_dict

In [26]:
def tracks_to_bars(song_dict: dict) -> dict:
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        if index < len(down_beats_array) - 1:
            end_time = down_beats_array[index+1] 
        else:
            end_time = down_beats_array[index] + 2
            
        guitar_bar = []
        drums_bar = []
        for guitar_note in guitar.notes:
            if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                guitar_bar.append(guitar_note)    

        for drum_note in drums.notes:
            if (drum_note.start >= start_time) and (drum_note.end < end_time):
                drums_bar.append(drum_note)
                  
        drums_bars_list.append(drums_bar)
        guitar_bars_list.append(guitar_bar)

    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict

In [36]:
def standardize_bars(song_name, list_of_bars, downbeats):
    """ 
    This function standardizes the timing of musical bars 
    so that each bar will start at the same time point.
    It gets a list of bars and the list of downbeats as inputs
    and returns a list of bars that all start with time = 1
    """
    
    for i in range(len(list_of_bars)):
        for j in range(len(list_of_bars[i])):
            if i == 0:
                list_of_bars[0][j].start = list_of_bars[0][j].start / downbeats[1]
                list_of_bars[0][j].end = list_of_bars[0][j].end / downbeats[1]

            list_of_bars[i][j].start = list_of_bars[i][j].start / downbeats[i]
            list_of_bars[i][j].end = list_of_bars[i][j].end / downbeats[i]

    return {'song_title': song_name,
            'list_of_bars': list_of_bars}

In [83]:
song = pretty_midi.PrettyMIDI('test_data/Another-One-Bites-The-Dust-1.mid')

In [117]:
song.instruments[-2].notes[

[Note(start=5.750000, end=5.755208, pitch=40, velocity=102),
 Note(start=5.875000, end=5.880208, pitch=40, velocity=122),
 Note(start=6.000000, end=6.005208, pitch=49, velocity=70),
 Note(start=6.000000, end=6.005208, pitch=35, velocity=109),
 Note(start=6.250000, end=6.255208, pitch=42, velocity=57),
 Note(start=6.500000, end=6.505208, pitch=42, velocity=85),
 Note(start=6.500000, end=6.505208, pitch=40, velocity=115),
 Note(start=6.500000, end=6.505208, pitch=35, velocity=112),
 Note(start=6.750000, end=6.755208, pitch=42, velocity=8),
 Note(start=7.000000, end=7.005208, pitch=42, velocity=90),
 Note(start=7.000000, end=7.005208, pitch=35, velocity=116),
 Note(start=7.250000, end=7.255208, pitch=42, velocity=26),
 Note(start=7.500000, end=7.505208, pitch=42, velocity=92),
 Note(start=7.500000, end=7.505208, pitch=40, velocity=117),
 Note(start=7.500000, end=7.505208, pitch=35, velocity=113),
 Note(start=7.750000, end=7.755208, pitch=42, velocity=4),
 Note(start=8.000000, end=8.005208

In [44]:
temp_dict = extract_guitar_and_drums('test_data/Another-One-Bites-The-Dust-1.mid')

In [7]:
temp_dict

{'title': 'Another-One-Bites-The-Dust-1',
 'down_beats': array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
         22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
         44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
         66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
         88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
        110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
        132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
        154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
        176., 178.]),
 'guitar': Instrument(program=28, is_drum=False, name="+FUNK GTR"),
 'drums': Instrument(program=0, is_drum=True, name="DRUMS")}

In [45]:
next_dict = tracks_to_bars(temp_dict)

In [29]:
next_dict['drum_bars']

[[],
 [],
 [Note(start=5.750000, end=5.755208, pitch=40, velocity=102),
  Note(start=5.875000, end=5.880208, pitch=40, velocity=122)],
 [Note(start=6.000000, end=6.005208, pitch=49, velocity=70),
  Note(start=6.000000, end=6.005208, pitch=35, velocity=109),
  Note(start=6.250000, end=6.255208, pitch=42, velocity=57),
  Note(start=6.500000, end=6.505208, pitch=42, velocity=85),
  Note(start=6.500000, end=6.505208, pitch=40, velocity=115),
  Note(start=6.500000, end=6.505208, pitch=35, velocity=112),
  Note(start=6.750000, end=6.755208, pitch=42, velocity=8),
  Note(start=7.000000, end=7.005208, pitch=42, velocity=90),
  Note(start=7.000000, end=7.005208, pitch=35, velocity=116),
  Note(start=7.250000, end=7.255208, pitch=42, velocity=26),
  Note(start=7.500000, end=7.505208, pitch=42, velocity=92),
  Note(start=7.500000, end=7.505208, pitch=40, velocity=117),
  Note(start=7.500000, end=7.505208, pitch=35, velocity=113),
  Note(start=7.750000, end=7.755208, pitch=42, velocity=4)],
 [Note

In [46]:
dict_guitar_bars = standardize_bars(next_dict['song_title'], next_dict['guitar_bars'], next_dict['down_beats'])
dict_drum_bars = standardize_bars(next_dict['song_title'], next_dict['drum_bars'], next_dict['down_beats'])

In [141]:
dict_guitar_bars['list_of_bars'][32]

[Note(start=1.000000, end=1.001302, pitch=67, velocity=96),
 Note(start=1.000000, end=1.001465, pitch=71, velocity=96),
 Note(start=1.000000, end=1.001546, pitch=64, velocity=96),
 Note(start=1.000000, end=1.001546, pitch=62, velocity=96),
 Note(start=1.003906, end=1.005127, pitch=62, velocity=96),
 Note(start=1.003906, end=1.005208, pitch=67, velocity=96),
 Note(start=1.003906, end=1.005208, pitch=64, velocity=96),
 Note(start=1.003906, end=1.005290, pitch=71, velocity=96),
 Note(start=1.007812, end=1.009115, pitch=67, velocity=96),
 Note(start=1.007812, end=1.009115, pitch=64, velocity=96),
 Note(start=1.007812, end=1.009196, pitch=71, velocity=96),
 Note(start=1.007812, end=1.009196, pitch=62, velocity=96),
 Note(start=1.011719, end=1.013184, pitch=71, velocity=110),
 Note(start=1.011719, end=1.013184, pitch=64, velocity=110),
 Note(start=1.011719, end=1.013672, pitch=67, velocity=110),
 Note(start=1.011719, end=1.013672, pitch=62, velocity=110),
 Note(start=1.015625, end=1.016113, 

In [83]:
def bars_to_track(modelled_drum_bars: list) -> pretty_midi:
    drums_bars_list = modelled_drum_bars
    drum_track = pretty_midi.PrettyMIDI()
    drum_track.instruments.append(pretty_midi.Instrument(0, is_drum=True, name=''))
    
    for bar in drums_bars_list:
        for note in bar:
            drum_track.instruments[0].notes.append(note)

    drum_track.write('test_drum_track.mid')

In [84]:
bars_to_track(next_dict['drum_bars'])

In [118]:
def bars_to_midi_bars(dict_guitar_bars: dict, dict_drum_bars: dict):
    guitar_bars_list = dict_guitar_bars['list_of_bars']
    for i, guitar_bar in enumerate(guitar_bars_list):
        guitar_bar_midi = pretty_midi.PrettyMIDI()
        guitar_bar_midi.instruments.append(pretty_midi.Instrument(28, is_drum=False, name='Guitar'))
        for note in guitar_bar:
            guitar_bar_midi.instruments[0].notes.append(note)
        if not os.path.exists(f"midi_bars/{dict_guitar_bars['song_title']}/guitar/"):
            os.makedirs(f"midi_bars/{dict_guitar_bars['song_title']}/guitar/")
        guitar_bar_midi.write(f"midi_bars/{dict_guitar_bars['song_title']}/guitar/bar_{i+1}.mid")

    drum_bars_list = dict_drum_bars['list_of_bars']
    for i, drum_bar in enumerate(drum_bars_list):
        drum_bar_midi = pretty_midi.PrettyMIDI()
        drum_bar_midi.instruments.append(pretty_midi.Instrument(0, is_drum=True, name='Drums'))
        for note in drum_bar:
            drum_bar_midi.instruments[0].notes.append(note)
        if not os.path.exists(f"midi_bars/{dict_guitar_bars['song_title']}/drums/"):
            os.makedirs(f"midi_bars/{dict_guitar_bars['song_title']}/drums/")
        drum_bar_midi.write(f"midi_bars/{dict_guitar_bars['song_title']}/drums/bar_{i+1}.mid")

In [119]:
bars_to_midi_bars(dict_guitar_bars, dict_drum_bars)

In [142]:
bar = pretty_midi.PrettyMIDI('midi_bars/Another-One-Bites-The-Dust-1/guitar/bar_33.mid')

In [144]:
bar.instruments[0].notes

[Note(start=1.000000, end=1.002273, pitch=62, velocity=96),
 Note(start=1.000000, end=1.002273, pitch=64, velocity=96),
 Note(start=1.000000, end=1.002273, pitch=67, velocity=96),
 Note(start=1.000000, end=1.002273, pitch=71, velocity=96),
 Note(start=1.004545, end=1.009091, pitch=62, velocity=96),
 Note(start=1.006818, end=1.009091, pitch=62, velocity=96),
 Note(start=1.004545, end=1.009091, pitch=64, velocity=96),
 Note(start=1.006818, end=1.009091, pitch=64, velocity=96),
 Note(start=1.004545, end=1.009091, pitch=67, velocity=96),
 Note(start=1.006818, end=1.009091, pitch=67, velocity=96),
 Note(start=1.004545, end=1.009091, pitch=71, velocity=96),
 Note(start=1.006818, end=1.009091, pitch=71, velocity=96),
 Note(start=1.011364, end=1.013636, pitch=62, velocity=110),
 Note(start=1.011364, end=1.013636, pitch=64, velocity=110),
 Note(start=1.011364, end=1.013636, pitch=67, velocity=110),
 Note(start=1.011364, end=1.013636, pitch=71, velocity=110),
 Note(start=1.015909, end=1.018182, 